# Использование доступных моделей
Данный ноутбук показывает базовый порядок работы с библиотекой.
- загрузка датасета
- деление на трейн и тест
- обучение модели
- сравнение с бейзлайном

Документацию можно собрать из папки `docs` командой `make html`. Она создаст папку `_build` с документацией -- `sponge-bob-magic/docs/_build/html/index.html`

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os, sys

parent_dir = os.path.split(os.getcwd())[0]
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

Целевая архитектура библиотеки - ЛД 3.0, поэтому используем Spark.
Для отладки моделей (не на кластере) создаём локальную сессию.

Объект `State` позволяет использовать одну и ту же сессию Spark в разных объектах.
Если сессия уже инициализированна, то её нужно передать при инициализации `State`. Модули библиотеки, которым необходимо использовать спарк-сессию, например для конвертации из пандас в спарк, будут искать сессию именно в `State`.

По умолчанию создастся дефолтная сессия. Простой способ получить сессию с заданным количеством выделенной памяти -- функция `get_spark_session` модуля `session_handler`.

In [3]:
from sponge_bob_magic.session_handler import State

spark = State().session
spark

## Подготовка данных <a name='data-preparator'></a>
Библиотека содержит утилиты для скачивания и парсинга популярных датасетов для рекомендаций.
Если датасет не найден в заданной директории, он будет автоматически скачан и обработан.

Данные датасета доступны в виде `pandas.DataFrame` атрибутов объекта.
Посмотреть доступные данные можно с помощью метода `info`.

In [4]:
from sponge_bob_magic.datasets import MovieLens

data = MovieLens("1m")
data.ratings.to_csv("data.csv", index=False)
data.info()

ratings


,user_id,item_id,relevance,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968



users


,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117



items


,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


Внутренний формат данных в либе -- спарк датафрейм. Сплиттеры умеют втоматически конвертировать пандас датафреймы, но модели сейчас ожидают именно спарк, причем с заданными обязательными колонками: `user_id, item_id, timestamp, relevance, context`. 

Получить данные в нужном формате можно с помощью `DataPreparator.transform_log`, который создаст нужные колонки, если их нет. Сейчас он умеет только читать с диска, поэтому на предыдущем шаге мы сохранили данные в виде csv.

In [5]:
from sponge_bob_magic.data_preparator import DataPreparator

log = DataPreparator(spark).transform_log(
    "data.csv",
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "relevance": "relevance",
        "timestamp": "timestamp"
    },
    header=True,
    format_type="csv"
)

Библиотека содержит различные схемы валидации рекомендательных систем, встречающиеся в литературе.

`UserSplitter` отбирает для теста некоторое количество или долю объектов для каждого пользователя.

In [6]:
from sponge_bob_magic.splitters import UserSplitter

splitter = UserSplitter(
    drop_cold_items=True,
    drop_cold_users=True,
    item_test_size=1,
    user_test_size=500,
    seed=1234,
    shuffle=True
)
train, test_input, test = splitter.split(log)
(
    train.count(), 
    test_input.count(), 
    test.count()
)

(999703, 999703, 506)

## NMF
Простейший пример использования DL в рекомендациях

In [9]:
from sponge_bob_magic.models import NeuroMFRec

nmf = NeuroMFRec(
    learning_rate=0.01,
    epochs=50,
    embedding_dimension=100
)

In [10]:
%%time

nmf.fit(
    log=train,
    user_features=None,
    item_features=None
)

02-Mar-20 11:59:58, root, DEBUG: Проверка датафреймов
02-Mar-20 12:00:00, root, DEBUG: Предварительная стадия обучения (pre-fit)
02-Mar-20 12:00:09, root, DEBUG: Основная стадия обучения (fit)
02-Mar-20 12:00:09, root, DEBUG: Индексирование данных
02-Mar-20 12:00:09, root, DEBUG: Составление батча:
02-Mar-20 12:00:09, root, DEBUG: -- Запись
02-Mar-20 12:00:10, root, DEBUG: -- Считывание
02-Mar-20 12:00:11, root, DEBUG: Обучение модели
02-Mar-20 12:00:11, root, DEBUG: -- Эпоха 0
02-Mar-20 12:00:31, root, DEBUG: -- Текущее значение train loss: 0.9341
02-Mar-20 12:00:32, root, DEBUG: -- Текущее значение val loss  : 0.7546
02-Mar-20 12:00:32, root, DEBUG: -- Эпоха 1
02-Mar-20 12:00:51, root, DEBUG: -- Текущее значение train loss: 0.4952
02-Mar-20 12:00:52, root, DEBUG: -- Текущее значение val loss  : 0.3329
02-Mar-20 12:00:52, root, DEBUG: -- Сохранение лучшей модели в файл
02-Mar-20 12:00:52, root, DEBUG: -- Эпоха 2
02-Mar-20 12:01:10, root, DEBUG: -- Текущее значение train loss: 0.2799
0

KeyboardInterrupt: 

In [9]:
%%time

recs = nmf.predict(
    log=train,
    k=10,
    users=test.select('user_id').distinct(),
    items=test.select('item_id').distinct(),
    context='no_context',
    user_features=None,
    item_features=None,
    filter_seen_items=True
)

CPU times: user 1.24 s, sys: 148 ms, total: 1.39 s
Wall time: 17.9 s


В библиотеке реализовано несколько различных метрик качества рекомендательных систем, встречающихся в литературе

In [10]:
from sponge_bob_magic.metrics import HitRate

hit_rate = HitRate()

hit_rate(recs, test, k=10)

0.039525691699604744

In [11]:
from sponge_bob_magic.metrics import NDCG

ndcg = NDCG()

ndcg(recs, test, k=10)

0.020361494493515788

## ALS
Библиотека также содержит классические алгоритмы рекомендаций, например, матричную факторизацию

In [12]:
from sponge_bob_magic.models.als_rec import ALSRec

als = ALSRec(rank=100)

In [13]:
%%time
als.fit(
    log=train,
    user_features=None,
    item_features=None
)

CPU times: user 24 ms, sys: 12 ms, total: 36 ms
Wall time: 32.1 s


In [14]:
%%time
recs = als.predict(
    k=10,
    users=test.select('user_id').distinct(),
    items=test.select('item_id').distinct(),
    context='no_context',
    log=train,
    user_features=None,
    item_features=None,
    filter_seen_items=True
)

CPU times: user 28 ms, sys: 8 ms, total: 36 ms
Wall time: 14.9 s


In [15]:
hit_rate(recs, test, k=10)

0.12055335968379446

In [16]:
ndcg(recs, test, k=10)

0.04913283620755998

## Своя модель
Для построения своей модели нужно использовать тот же самый split, что и для бейзлайнов

In [17]:
train.toPandas().to_csv("train.csv", index=False)

In [18]:
!head train.csv

user_id,item_id,relevance,timestamp,context
1090,2433,3.0,2000-11-23 01:26:01,no_context
1090,1247,4.0,2000-11-23 01:21:02,no_context
1090,954,4.0,2000-11-23 01:21:50,no_context
1090,3386,3.0,2000-11-23 01:14:07,no_context
1090,1939,3.0,2000-11-23 01:20:32,no_context
1090,2428,2.0,2000-11-23 01:27:11,no_context
1090,1673,3.0,2000-11-23 01:05:15,no_context
1090,1645,4.0,2000-11-23 01:33:11,no_context
1090,1093,3.0,2000-11-23 01:26:44,no_context


Здесь магия: можно взять train и обучить на нём свою любимую модель вне библиотеки

Также нужно выдать рекомендации обученной моделью

In [19]:
recs.toPandas().to_csv("recs.csv", index=False)

Теперь нужно прочитать рекомендации в формате, поддерживаемом библиотекой

In [20]:
recs = DataPreparator(spark).transform_log(
    "recs.csv",
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "relevance": "relevance"
    },
    header=True,
    format_type="csv"
)

и сравнить качество своей модели с бейзлайнамии

In [21]:
hit_rate(recs, test, k=10)

0.12055335968379446